<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>
<br>
<h1 align=center><font size = 5>IBM Data Science Capstone - Eugene Bible</font></h1>

# Introduction

This is the Jupyter Notebook to be used for the final project in the IBM Data Science Capstone course of the IBM Data Science Professional Certificate program by Coursera.

In this Jupyter notebook, neighborhoods in the city of Toronto will be explored, segmented, and clustered. 
First, the data will be scraped from the Wikipedia page and be wrangled, cleaned, and then read into a pandas dataframe in a structured format. Once structured, the neighborhoods in Toronoto will be explored and clustered.

Author: Eugene Bible

This project link on IBM Watson Studio: https://jp-tok.dataplatform.cloud.ibm.com/analytics/notebooks/v2/007ab5db-57d4-4ac7-b5fd-08ab9d2e0afc/view?projectid=0e052036-fd3e-4893-a04b-46349d1af3d9&context=analytics

This project link on Github: https://github.com/Glitch852/Coursera_Capstone/blob/master/IBM%20Data%20Science%20Capstone%20-%20EB.ipynb

## Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

[1. Segmenting and Clustering Neighborhoods in Toronto](#1.-Segmenting-and-Clustering-Neighborhoods-in-Toronto) <br>
[2. The Battle of Neighborhoods - Part 1](#2.-The-Battle-of-Neighborhoods-\--Part-1)<br>
[3. The Battle of Neighborhoods - Part 2](#3.-The-Battle-of-Neighborhoods-\--Part-2)


</font>
</div>

Import the necessary libraries:

In [2]:
import urllib.request,urllib.parse,urllib.error # Library to parse websites
from bs4 import BeautifulSoup # Library to handle web scraping

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder for getting lat/lon

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import json # library to handle json files

import requests #library to handle requests
from pandas.io.json import json_normalize #transform JSON file into pandas dataframe

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

# 1. Segmenting and Clustering Neighborhoods in Toronto

## Part 1.1 - Scraping the Data and Creating the Dataframe

### Scrape the Wikipedia page to obtain the table of postal codes/boroughs/neighborhoods

Page found at: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
# Define the URL of the webpage to scrape
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Get HTML
html = urllib.request.urlopen(url).read()
#print(html)

# Use BeautifulSoup to parse the html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

# Parse the table only and pull all rows
table = soup.find('table')
#print(table.prettify())

### Transform the table into a Pandas Dataframe
Requirements:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [4]:
# Scraping the rows of the table
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    #print('count = ' + str(count))
    th = tr.find_all('th') # Grab header
    td = tr.find_all('td') # Grab row
    if th:
        row = [th.text.rstrip() for th in th]
        rows.append(row)
    else:
        row = [tr.text.rstrip() for tr in td]
        if row[1] == 'Not assigned': continue # If no borough, don't keep the row
        if row[2] == 'Not assigned': row[2] = row[1] # If neighborhood is not assigned, set it to the Borough
        rows.append(row)  
            
# Assign list of rows to a Pandas DF
nhScraped_df = pd.DataFrame(rows)

# Name the columns after the first row (headers) then drop the header row
nhScraped_df.columns = nhScraped_df.iloc[0]
nhScraped_df = nhScraped_df.drop(nhScraped_df.index[0])
nhScraped_df.head(10)


,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Downtown Toronto,Queen's Park
7,M9A,Queen's Park,Queen's Park
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
10,M3B,North York,Don Mills North


In [5]:
# Let's check our counts of postal codes, boroughs, and neighborhoods.

print('The dataframe has {} postal codes, {} boroughs, and {} neighborhoods.'.format(
        len(nhScraped_df['Postcode'].unique()),
        len(nhScraped_df['Borough'].unique()),
        len(nhScraped_df['Neighbourhood'].unique())
    )
)

The dataframe has 103 postal codes, 11 boroughs, and 207 neighborhoods.


In [6]:
# Now that we have the 'Not Assigned' entries taken care of, need to combine the rows with the same Postcode but different neighborhoods

# Use groupby to sort the list by Postcode
nhGrouped_df = nhScraped_df.groupby(['Postcode','Borough'])
#print(nhGrouped_df)

# Print to check for proper sorting
for key, item in nhGrouped_df:
    print(nhGrouped_df.get_group(key), "\n\n")

0 Postcode      Borough Neighbourhood
8      M1B  Scarborough         Rouge
9      M1B  Scarborough       Malvern 


0  Postcode      Borough   Neighbourhood
21      M1C  Scarborough  Highland Creek
22      M1C  Scarborough      Rouge Hill
23      M1C  Scarborough      Port Union 


0  Postcode      Borough Neighbourhood
33      M1E  Scarborough     Guildwood
34      M1E  Scarborough   Morningside
35      M1E  Scarborough     West Hill 


0  Postcode      Borough Neighbourhood
39      M1G  Scarborough        Woburn 


0  Postcode      Borough Neighbourhood
43      M1H  Scarborough     Cedarbrae 


0  Postcode      Borough        Neighbourhood
54      M1J  Scarborough  Scarborough Village 


0  Postcode      Borough         Neighbourhood
66      M1K  Scarborough  East Birchmount Park
67      M1K  Scarborough               Ionview
68      M1K  Scarborough          Kennedy Park 


0  Postcode      Borough Neighbourhood
79      M1L  Scarborough      Clairlea
80      M1L  Scarborough   Gold

0  Postcode     Borough  Neighbourhood
92      M9L  North York  Humber Summit 


0   Postcode     Borough Neighbourhood
106      M9M  North York         Emery
107      M9M  North York     Humberlea 


0   Postcode Borough Neighbourhood
116      M9N    York        Weston 


0   Postcode    Borough Neighbourhood
126      M9P  Etobicoke     Westmount 


0   Postcode    Borough         Neighbourhood
137      M9R  Etobicoke     Kingsview Village
138      M9R  Etobicoke  Martin Grove Gardens
139      M9R  Etobicoke      Richview Gardens
140      M9R  Etobicoke          St. Phillips 


0   Postcode    Borough     Neighbourhood
174      M9V  Etobicoke    Albion Gardens
175      M9V  Etobicoke  Beaumond Heights
176      M9V  Etobicoke        Humbergate
177      M9V  Etobicoke         Jamestown
178      M9V  Etobicoke       Mount Olive
179      M9V  Etobicoke       Silverstone
180      M9V  Etobicoke     South Steeles
181      M9V  Etobicoke       Thistletown 


0   Postcode    Borough Neighbour

In [7]:
# Join neighborhoods of the grouped entries (i.e. the groups with the same postcode), separated by commas
nhJoined_df = nhGrouped_df['Neighbourhood'].apply(', '.join)

nhJoined_df = nhJoined_df.reset_index()

nhJoined_df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
# Check the number of rows in our df, should still have 103 postal codes, meaning 103 rows now.
nhJoined_df.shape

(103, 3)

### Getting latitude and longitude for each neighborhood
Now that we have a dataframe of the postal codes of each neighborhood, borough name, and neighborhood name, to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [54]:
## Using Python's built-in geocode NOTE: TOO UNRELIABLE. USED THE GEOSPATIAL_DATA PAGE PROVIDED BY INSTRUCTOR.
#
## initialize your variable to None
#lat_lng_coords = None
#
#address = 'M1J Scarborough Scarborough Village'
#geolocator = Nominatim(user_agent="EB_explores_canada")
#
#while(lat_lng_coords is None):
#    lat_lng_coords = geolocator.geocode(address)
#
#print(lat_lng_coords)
#    
#nlat = lat_lng_coords.latitude
#nlon = lat_lng_coords.longitude
#
#
#print(nlat, nlon)

Note: Package was too unreliable to get all lat/lon values. Used the CSV file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [9]:
url = 'http://cocl.us/Geospatial_data'
response = urllib.request.urlopen(url)
latlng_df = pd.read_csv(response)


latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Change column name so both dataframes have the same column name
latlng_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
latlng_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Merge two Dataframes on column 'Postcode'
neighborhoods_df = nhJoined_df.merge(latlng_df, on='Postcode')

neighborhoods_df = neighborhoods_df.rename(columns={'Neighbourhood': 'Neighborhood'}) # Because I'm American. (Sorry.)

neighborhoods_df.head(100)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
# Let's check our neighborhood and borough counts. Should still have 103 postal codes and 11 boroughs.
print('The dataframe has {} postal codes and {} boroughs.'.format(
        len(neighborhoods_df['Postcode'].unique()),
        len(neighborhoods_df['Borough'].unique())
    )
)

The dataframe has 103 postal codes and 11 boroughs.


## Part 1.2 - Analysis

## Exploring and clustering the groups of Toronto
Let's start by getting a map of the area.

In [14]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Canada are 43.653963, -79.387207.


ONLY BOROUGHS THAT CONTAIN TORONTO VERSION:

In [72]:
#toronto_df = neighborhoods_df.loc[(neighborhoods_df['Borough'].str.contains('Toronto'))]

#print(toronto_df)

In [61]:
# create map of Toronto using latitude and longitude values
#toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11.5)

## add markers to map
#for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
#    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(toronto_map)  
#    
#toronto_map

ALL OF TORONTO VERSION:

In [15]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Borough'], neighborhoods_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Define Foursquare credentials and version

In [16]:
CLIENT_ID = 'NINUZNGSTWMF0JOMP2EJVFJA5Z33VY5CQCZZD3YW4MBTHSIM' # your Foursquare ID
CLIENT_SECRET = 'P1SHG0HE4PAAJXCEG5DOENHIMP4DRYEL2GLZZ2IMQSAO1B3H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# Set the number of venues from each borough
LIMIT = 100


Your credentials:
CLIENT_ID: NINUZNGSTWMF0JOMP2EJVFJA5Z33VY5CQCZZD3YW4MBTHSIM
CLIENT_SECRET:P1SHG0HE4PAAJXCEG5DOENHIMP4DRYEL2GLZZ2IMQSAO1B3H


Next, let's create a function to get all the venues within 500m of each neighborhood in Toronto.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# Let's call the function and see what kind of venues we get
toronto_venues = getNearbyVenues(names=neighborhoods_df['Borough'],
                                   latitudes=neighborhoods_df['Latitude'],
                                   longitudes=neighborhoods_df['Longitude']
                                  )

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2213, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Scarborough,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,115,115,115,115,115,115
Downtown Toronto,1296,1296,1296,1296,1296,1296
East Toronto,122,122,122,122,122,122
East York,77,77,77,77,77,77
Etobicoke,74,74,74,74,74,74
Mississauga,12,12,12,12,12,12
North York,242,242,242,242,242,242
Scarborough,84,84,84,84,84,84
West Toronto,173,173,173,173,173,173


In [21]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 unique categories.


One Hot Encoding

In [39]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# Let's look at all the different columns
cols = list(toronto_onehot)
#print(cols)

# Move the neighborhood column to the front of the list
cols.insert(0, cols.pop(cols.index('Neighborhood')))
#print(cols)

# Re-order the toronto_onehot df according to the re-ordered columns list
toronto_onehot = toronto_onehot.loc[toronto_onehot.index[:], cols]
toronto_onehot.head()

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
toronto_onehot.shape

(2213, 266)

Let's group rows by neighborhood and take the mean of the frequency of occurance for each category

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017391,0.000000,0.000000,0.000000,0.000000,0.008696,0.000000,0.000000,0.008696,0.000000,0.000000,0.000000,0.000000,0.

In [42]:
toronto_grouped.shape

(10, 266)

# 2. The Battle of Neighborhoods - Part 1

## Part 2.1 - 

## Part 2.2 - 

# 3. The Battle of Neighborhoods - Part 2

## Part 3.1 -

## Part 3.2 -